In [2]:
import sys
sys.path.append('..')

import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torchvision.datasets import CIFAR10

In [5]:
def vgg_block(num_convs,in_channels,out_channels):
    net=[nn.Conv2d(in_channels,out_channels,kernel_size=3,
                  padding=1),nn.ReLU(True)]
    for i in range(num_convs-1):
        net.append(nn.Conv2d(out_channels,out_channels,kernel_size=3,padding=1))
        net.append(nn.ReLU(True))
    
    net.append(nn.MaxPool2d(2,2))
    return nn.Sequential(*net)

In [6]:
block_demo=vgg_block(3,64,128)
print(block_demo)

Sequential(
  (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): ReLU(inplace)
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [7]:
input_demo=Variable(torch.zeros(1,64,300,300))
output_demo=block_demo(input_demo)
print(output_demo.shape)

torch.Size([1, 128, 150, 150])


In [8]:
def vgg_stack(num_convs,channels):
    net=[]
    for n,c in zip(num_convs,channels):
        in_c=c[0]
        out_c=c[1]
        net.append(vgg_block(n,in_c,out_c))
    return nn.Sequential(*net)

In [9]:
vggnet=vgg_stack((1,1,2,2,2),((3,64),(64,128),(128,256),(256,512),(512,512)))
print(vggnet)

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (3): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=

In [10]:
test_x=Variable(torch.zeros(1,3,256,256))
test_y=vggnet(test_x)
print(test_y.shape)

torch.Size([1, 512, 8, 8])


In [15]:
class vgg(nn.Module):
    def __init__(self):
        super(vgg,self).__init__()
        self.feature=vggnet
        self.fc=nn.Sequential(
        nn.Linear(512,100),
        nn.ReLU(True),
        nn.Linear(100,10))
    def forward(self,x):
        x=self.feature(x)
        x=x.view(x.shape[0],-1)
        x=self.fc(x)
        return x

In [17]:
from utils import train

def data_tf(x):
    x=np.array(x,dtype='float32')/255
    x=(x-0.5)/0.5
    x=x.transpose((2,0,1))
    x=torch.from_numpy(x)
    return x 

train_set=CIFAR10('./data',train=True,transform=data_tf,download=True)
train_data=torch.utils.data.DataLoader(train_set,batch_size=64,shuffle=True)
test_set=CIFAR10('./data',train=False,transform=data_tf,download=True)
test_data=torch.utils.data.DataLoader(train_set,batch_size=128,shuffle=False)

net=vgg()
optimizer=torch.optim.SGD(net.parameters(),lr=1e-1)
criterion=nn.CrossEntropyLoss()


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
train(net, train_data, test_data, 20, optimizer, criterion)

C:\Users\TEMP\Desktop\Pytorch\utils.py:58: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  im = Variable(im, volatile=True)
C:\Users\TEMP\Desktop\Pytorch\utils.py:59: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  label = Variable(label, volatile=True)


Epoch 0. Train Loss: 2.303064, Train Acc: 0.099984, Valid Loss: 2.302851, Valid Acc: 0.100012, Time 00:30:02
Epoch 1. Train Loss: 2.302959, Train Acc: 0.098765, Valid Loss: 2.302715, Valid Acc: 0.100012, Time 00:39:55
Epoch 2. Train Loss: 2.302883, Train Acc: 0.099604, Valid Loss: 2.302371, Valid Acc: 0.099964, Time 00:38:35
Epoch 3. Train Loss: 2.294750, Train Acc: 0.116208, Valid Loss: 2.266562, Valid Acc: 0.129132, Time 00:38:17
Epoch 4. Train Loss: 1.939736, Train Acc: 0.255555, Valid Loss: 2.097532, Valid Acc: 0.245996, Time 00:49:45
Epoch 5. Train Loss: 1.583835, Train Acc: 0.398557, Valid Loss: 1.464855, Valid Acc: 0.455523, Time 00:39:26
Epoch 6. Train Loss: 1.361566, Train Acc: 0.493386, Valid Loss: 1.215324, Valid Acc: 0.549365, Time 01:36:54
